In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
print(len(text))
print(text[:200])


232310
﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars=sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
encoded_hello=encode("hello")
decoded_hello=decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [5]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 61, 54, 72, 73, 58, 67, 58],
        [61, 58,  1, 75, 58, 60, 58, 73],
        [ 1, 61, 54, 75, 58,  1, 61, 54],
        [54, 57,  1, 57, 68, 67, 58,  1]])
targets:
tensor([[61, 54, 72, 73, 58, 67, 58, 57],
        [58,  1, 75, 58, 60, 58, 73, 54],
        [61, 54, 75, 58,  1, 61, 54, 57],
        [57,  1, 57, 68, 67, 58,  1, 58]])


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


vZS6V.Iv*83hdT5VYtSvYUN[,[d;H'U0PgRvae"pcP3cWZbhbhb05U,bcQoH?Gt0pcFwjpl!cqng9*p-j-Hzw2JNHhHrBYHgl2h
Q-q7X-qU 5,Uvc6vU 6Z-Gt]ZN'1Mb
5I'R;JX﻿yf aFj)fVYI;:E9:edJdvUtTiZ6n:]lvLv_YJmZ,&[GHN
BzOX_hH
;u"rv[tR;tdw:] prw
w1j6Vdr[_1Y5)FmnbYUx!Gi[,7kT715wo&(7tRM'OPO
XzIwqMaw!cq,7iEd73py.H61.Hc2X*d 9m4"C[rwJg9OppcWb1!khJY:Z;DXGHr)xWKbpii"e&(
AGP(Q82!V _x!8&mxI;V&K[;?D6l,"j&1rb(1XUuoC9zTxKQTvkPLK6;syt﻿TN.OZ
;DZ!fQpcHa(P:ICWiwR;0?)Y5H!Q-(qROwT
﻿HTxM2B*oNqVphb:WimBl'iv6zHoXV!f,JN.o,zA,qVu32IHquOg7u1j:8b]&?4
Xx


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.962, val loss: 4.963
step: 250, train loss: 4.920, val loss: 4.912
step: 500, train loss: 4.861, val loss: 4.852
step: 750, train loss: 4.772, val loss: 4.781
5.007771968841553


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


B&'x!F;EX!X*hbcoSt]oWJ(m(50ZrtdnX*0﻿1(aT3RbzjSZ.uF*J9QB&VK5d 7g;fuPQ-8'﻿Dtl]t"Iy_0e

11S.0
1NYv[vtT_EB7N!jnT&cM)HpNxGBh"8xMx.Z[OwojOAXDHN8dX:6VlT5dXbR;h"(s(_)04!o2d rFdX!Uyu9fnAbc8itc)IEqVw:]LUn6bjt
RRZ,v)CW:_kHhZ;f(6[0(;pME&klyj:]H9Q-l7r﻿HTuaKDJ2B2C1HO _5dThDd0"kcH]H!'RZ_'48X_rk30Zd
ys_maga﻿JgBMEO0ZjSK;2nBgTx4
xNcyj3eCB :ry,N[*R;'1z ? fVYSP?IidMHp!zTX!BzK8:)
lxS"TG"b
Y!cTX1xVY6*iCN
"SocfD_LYM:50e_mN42)&urBp,7SDMAFREae&GdV!k*(mu]HePkT.J nmWuIb(Z0)j]nd,oxsSDm mu]3,WFmqas1v*Jgm5EZ-DY!,WuC)xVYhB7?c


##### 1. Mean Squared Error (MSE): 
MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.

##### 2.Gradient Descent (GD): 
is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function

##### 3.Momentum: 
Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.

##### 4.RMSprop: 
RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.

##### 5.Adam: 
Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.

##### 6. AdamW: 
AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.